In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 58.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=0e95d147fcf0d9c011defdec51b878db609887a1e6abf7a2a4e530144e0394ac
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


# 1. Read the given CSV file in a Hive table


In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
appName= "bosten"
master= "local"

In [4]:
df=pd.read_csv("/content/drive/MyDrive/boston.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CRIM    506 non-null    float64
 1   ZN      506 non-null    float64
 2   INDUS   506 non-null    float64
 3   CHAS    506 non-null    int64  
 4   NOX     506 non-null    float64
 5   RM      506 non-null    float64
 6   AGE     506 non-null    float64
 7   DIS     506 non-null    float64
 8   RAD     506 non-null    int64  
 9   TAX     506 non-null    int64  
 10  PT      506 non-null    float64
 11  B       506 non-null    float64
 12  LSTAT   506 non-null    float64
 13  MV      506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


In [6]:
spark = SparkSession.builder \
	.master(master).appName("appName").enableHiveSupport().getOrCreate()

In [7]:
spark

In [8]:
df=spark.sql("show databases")
df.show()

+---------+
|namespace|
+---------+
|  default|
+---------+



#Q2. Read the data from Hive table as spark dataframe


In [24]:
df=spark.read.option('header','true').csv('/content/drive/MyDrive/boston.csv',inferSchema=True)

In [10]:
df.show()

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|18.0|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|       24.0|
|0.027310001| 0.0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729| 0.0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001| 0.0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999| 0.0|2.

In [11]:
df.printSchema()

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PT: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MV: double (nullable = true)



# Q3. Get the correlation between dependent and independent variables

In [12]:
import six
for i in df.columns:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to MV for ", i, df.stat.corr('MV',i))

Correlation to MV for  CRIM -0.3883046116575088
Correlation to MV for  ZN 0.36044534463752903
Correlation to MV for  INDUS -0.48372517128143383
Correlation to MV for  CHAS 0.17526017775291847
Correlation to MV for  NOX -0.4273207763683772
Correlation to MV for  RM 0.695359937127267
Correlation to MV for  AGE -0.37695456714288667
Correlation to MV for  DIS 0.24992873873512172
Correlation to MV for  RAD -0.3816262315669168
Correlation to MV for  TAX -0.46853593528654536
Correlation to MV for  PT -0.5077867038116085
Correlation to MV for  B 0.3334608226834164
Correlation to MV for  LSTAT -0.7376627294671615
Correlation to MV for  MV 1.0


In [13]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')


In [14]:
df1 = vectorAssembler.transform(df)

# Q4. Build a linear regression model to predict house price

In [15]:
splits = df1.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [16]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-0.043530258212475144,0.0,-0.01822690203057224,1.2848866908936603,-7.346598467968144,5.110374360093858,0.0,-0.5817889396040057,0.0,-0.004347981340920702,-0.8451841840268647,0.006983489236734514,-0.37541195852286596]
Intercept: 16.379321259831173


# Q5. Evaluate the Linear Regression model by getting the RMSE and R-squared values

In [17]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 4.243667
r2: 0.778388


In [18]:
train_df.describe().show()

+-------+------------------+------------------+-----------------+------------------+-------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+
|summary|              CRIM|                ZN|            INDUS|              CHAS|                NOX|                RM|              AGE|               DIS|              RAD|               TAX|               PT|                 B|             LSTAT|                MV|
+-------+------------------+------------------+-----------------+------------------+-------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+
|  count|               362|               362|              362|               362|                362|               362|              362|               362|              362|   

In [19]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","MV","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
| 30.28954032965682|       24.0|[0.00632,18.0,2.3...|
|30.229154019432812|35.40000153|[0.01311,90.0,1.2...|
|38.586833122870914|       50.0|[0.01381,80.0,0.4...|
|26.471504284588185|30.10000038|[0.01709,90.0,2.0...|
| 30.29808543378417|32.90000153|[0.01778,95.0,1.4...|
+------------------+-----------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.543424


In [20]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 6.48899


In [21]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 10
objectiveHistory: [0.5, 0.4272399962663321, 0.2047649174485246, 0.18156620696863232, 0.15168886183481112, 0.14999518317887792, 0.1495172124977353, 0.1490054463133159, 0.14870360886947068, 0.1486626313744547, 0.14848991743841428]
+-------------------+
|          residuals|
+-------------------+
| 1.2037748540520994|
| -5.637116623657008|
|0.32602862920415276|
| 2.4318594727885348|
| 0.8902078837428249|
|-1.0590092717455803|
|-2.5949915161168633|
|  9.018416397728451|
|   9.14666696749071|
|  5.700751452760699|
|-1.0688490762035023|
| 10.400107132778487|
|-0.5501235172245202|
|0.06728561700109381|
|  2.934126456399717|
|  -4.37161504776406|
|  2.480839819619135|
|-1.1830206384181494|
| 0.6460393994252165|
| -1.604945716074404|
+-------------------+
only showing top 20 rows



In [22]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","MV","features").show()

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
| 30.28954032965682|       24.0|[0.00632,18.0,2.3...|
|30.229154019432812|35.40000153|[0.01311,90.0,1.2...|
|38.586833122870914|       50.0|[0.01381,80.0,0.4...|
|26.471504284588185|30.10000038|[0.01709,90.0,2.0...|
| 30.29808543378417|32.90000153|[0.01778,95.0,1.4...|
|  25.1826024227356|23.10000038|[0.0187,85.0,4.15...|
|36.190963620724716|42.29999924|[0.02177,82.5,2.0...|
| 26.51677372465162|       16.5|[0.024979999,0.0,...|
|27.230051776051972|23.89999962|[0.025429999,55.0...|
|26.332570097090798|28.70000076|[0.029850001,0.0,...|
|31.389646649251134|34.90000153|[0.03359,75.0,2.9...|
| 20.49516507242332|       19.5|[0.03427,0.0,5.19...|
|29.582393469661096|       28.5|[0.035020001,80.0...|
| 38.34070072869348|       48.5|[0.035100002,95.0...|
| 27.63598491846548|       23.5|[0.035840001,80.0...|
| 23.12730569962929|20.70000

In [23]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'MV')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="MV", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 5.58283
